# ET0203 – Seminario de la Ciencia de los Datos
## Unidad 1 · EDA con Kaggle – Titanic Dataset

**Estudiante:** Palacio Manuela  
**Código:** 1023774044  
**Dataset:** Titanic — Machine Learning from Disaster  
**Enlace:** https://www.kaggle.com/competitions/titanic  
**Fecha:** 2026-02-26  

---

## 1) Preparación del entorno

In [ ]:
# ============================================================
# 1) Preparación
# ============================================================
# pip install pandas numpy matplotlib seaborn scipy scikit-learn statsmodels missingno openpyxl

import os
import re
import math
import json
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import missingno as msno

from scipy import stats
from scipy.stats.mstats import winsorize

from sklearn.preprocessing import PowerTransformer
from sklearn.cluster import DBSCAN
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor

import statsmodels.api as sm
from statsmodels.stats.anova import anova_lm

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 140)
plt.rcParams["figure.figsize"] = (10, 5)
sns.set_style("whitegrid")

print("Entorno listo.")

## 2) Asignación automática del dataset

In [ ]:
# ============================================================
# 2) Asignación automática (usa la lista oficial del curso)
# ============================================================
RUTA_LISTA = r"ET0203 - SEMINARIO DE LA CIENCIA DE LOS DATOS L-W.xlsx"

# Catálogo de datasets Kaggle (6 opciones)
DATASETS = [
    dict(id="DS1", nombre="Titanic — Machine Learning from Disaster", tipo="Competencia",
         url="https://www.kaggle.com/competitions/titanic"),
    dict(id="DS2", nombre="House Prices — Advanced Regression Techniques", tipo="Competencia",
         url="https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques"),
    dict(id="DS3", nombre="Medical Cost Personal Datasets (Insurance)", tipo="Dataset",
         url="https://www.kaggle.com/datasets/mirichoi0218/insurance"),
    dict(id="DS4", nombre="Students Performance in Exams", tipo="Dataset",
         url="https://www.kaggle.com/datasets/spscientist/students-performance-in-exams"),
    dict(id="DS5", nombre="Walmart Dataset (ventas)", tipo="Dataset",
         url="https://www.kaggle.com/datasets/yasserh/walmart-dataset"),
    dict(id="DS6", nombre="Superstore Sales Dataset (Sales Forecasting)", tipo="Dataset",
         url="https://www.kaggle.com/datasets/rohitsahoo/sales-forecasting"),
]

def asignar_datasets_por_codigo(codigo: str, k=3):
    digits = re.sub(r"\D", "", str(codigo))
    seed = int(digits[-9:]) if digits else abs(hash(str(codigo))) % (2**31-1)
    rng = np.random.default_rng(seed)
    idx = rng.choice(len(DATASETS), size=k, replace=False)
    return [DATASETS[i] for i in idx]

try:
    est = pd.read_excel(RUTA_LISTA, sheet_name=0)
    est["Codigo"] = est["Codigo"].astype(str).str.strip()
    rows = []
    for _, row in est.iterrows():
        codigo = row["Codigo"]
        picks = asignar_datasets_por_codigo(codigo, k=3)
        rows.append({"Codigo": codigo, "URL_Principal": picks[0]["url"]})
    asignacion = pd.DataFrame(rows).sort_values("Codigo").reset_index(drop=True)
    pd.set_option("display.max_colwidth", None)
    display(asignacion.head(30))
except FileNotFoundError:
    print("Nota: Archivo de lista de estudiantes no encontrado. Se usa asignación directa.")
    asignacion = None

In [ ]:
# Mi asignación
MI_CODIGO = "1023774044"

# Verificar asignación
picks = asignar_datasets_por_codigo(MI_CODIGO, k=3)
print(f"Código: {MI_CODIGO}")
print(f"Dataset principal: {picks[0]['nombre']}")
print(f"URL: {picks[0]['url']}")
print(f"\nAlternativos:")
for p in picks[1:]:
    print(f"  - {p['nombre']}: {p['url']}")

# Verificar en tabla de asignación si existe
if asignacion is not None:
    fila = asignacion.loc[asignacion["Codigo"] == str(MI_CODIGO).strip()]
    if not fila.empty:
        display(fila)

## 3) Descarga del dataset desde Kaggle

**Dataset asignado:** Titanic — Machine Learning from Disaster  
**Enlace:** https://www.kaggle.com/competitions/titanic  
**Descarga:** Manual desde Kaggle → se colocó `train.csv` en la carpeta `data/`  
**Archivo principal:** `train.csv` (891 filas, 12 columnas)

In [ ]:
# Descarga manual: se descargó train.csv desde Kaggle y se colocó en data/
# Alternativa con API:
# !kaggle competitions download -c titanic -p data/ --force
# import zipfile
# with zipfile.ZipFile("data/titanic.zip", "r") as z:
#     z.extractall("data/")

## 4) Carga del archivo y comprensión del contexto

### Contexto del negocio
El RMS Titanic se hundió el 15 de abril de 1912 tras chocar con un iceberg. De 2224 pasajeros y tripulación, murieron 1502. Este dataset contiene información de 891 pasajeros del set de entrenamiento.

### Unidad de observación
Cada fila representa **un pasajero** del Titanic.

### Diccionario de datos

| Variable | Tipo | Significado | Rango esperado |
|----------|------|------------|----------------|
| PassengerId | int | Identificador único | 1–891 |
| Survived | int (binaria) | Sobrevivió (1) o no (0) | 0, 1 |
| Pclass | int (ordinal) | Clase del boleto (proxy de estatus socioeconómico) | 1, 2, 3 |
| Name | string | Nombre completo del pasajero | texto libre |
| Sex | string | Género del pasajero | male, female |
| Age | float | Edad en años | 0.42–80 |
| SibSp | int | # de hermanos/cónyuge a bordo | 0–8 |
| Parch | int | # de padres/hijos a bordo | 0–6 |
| Ticket | string | Número de ticket | texto libre |
| Fare | float | Tarifa pagada en libras | 0–512.33 |
| Cabin | string | Número de cabina | texto libre (77% faltante) |
| Embarked | string | Puerto de embarque | C=Cherbourg, Q=Queenstown, S=Southampton |

### Variables clave para el análisis
- **Target:** Survived
- **Numéricas continuas:** Age, Fare
- **Numéricas discretas:** SibSp, Parch, Pclass
- **Categóricas:** Sex, Embarked, Cabin

### Supuestos y riesgos
- PassengerId es solo identificador, no tiene valor analítico
- Age tiene ~20% de valores faltantes
- Cabin tiene ~77% faltante — difícil de usar directamente
- Fare=0 podría ser dato erróneo o tripulación

In [ ]:
# ============================================================
# 4) Carga
# ============================================================
os.makedirs("data", exist_ok=True)
os.makedirs("exports/figuras", exist_ok=True)

RUTA_CSV = "data/train.csv"

df = pd.read_csv(RUTA_CSV)
print(f"Dataset cargado: {df.shape[0]} filas x {df.shape[1]} columnas")
df.head(10)

In [ ]:
print("Shape:", df.shape)
print("\n--- Info ---")
df.info()
print("\n--- Describe (todas las columnas) ---")
df.describe(include="all").T

**Conclusión Sección 4:** El dataset tiene 891 pasajeros y 12 variables. Las variables numéricas continuas más relevantes son Age y Fare. Se identifica que Age tiene 177 valores faltantes (19.87%) y Cabin 687 (77.1%). La variable Survived es la variable objetivo (38.4% sobrevivieron).

## 5) Auditoría de calidad

In [ ]:
# ============================================================
# 5) Auditoría de calidad
# ============================================================

# --- 5.1 Faltantes ---
def resumen_faltantes(data: pd.DataFrame) -> pd.DataFrame:
    na = data.isna().sum()
    pct = (na / len(data) * 100).round(2)
    return pd.DataFrame({"faltantes": na, "pct": pct}).sort_values("pct", ascending=False)

faltantes = resumen_faltantes(df)
print("=== Resumen de valores faltantes ===")
display(faltantes)

# --- 5.2 Duplicados ---
dup = df.duplicated().sum()
print(f"\nDuplicados (filas completas): {dup}")

# --- 5.3 Matriz de faltantes ---
msno.matrix(df.sample(min(len(df), 2000), random_state=42))
plt.title("Matriz de valores faltantes")
plt.show()

# --- 5.4 Columnas candidatas a fecha ---
posibles_fecha = [c for c in df.columns if re.search(r"date|fecha|time|timestamp", c, re.I)]
print("Columnas candidatas a fecha:", posibles_fecha if posibles_fecha else "Ninguna")

In [ ]:
# --- 5.5 Verificación de tipos de datos ---
print("=== Tipos de datos ===")
print(df.dtypes)

# --- 5.6 Rangos imposibles ---
print("\n=== Verificación de rangos ===")
print(f"Age min: {df['Age'].min()}, max: {df['Age'].max()} -> {'OK' if df['Age'].min() >= 0 else 'RANGO IMPOSIBLE'}")
print(f"Fare min: {df['Fare'].min()}, max: {df['Fare'].max()} -> {'Fare=0 sospechoso' if df['Fare'].min() == 0 else 'OK'}")
print(f"SibSp min: {df['SibSp'].min()}, max: {df['SibSp'].max()}")
print(f"Parch min: {df['Parch'].min()}, max: {df['Parch'].max()}")
print(f"Survived valores únicos: {sorted(df['Survived'].unique())} -> {'OK' if set(df['Survived'].unique()) == {0,1} else 'ERROR'}")
print(f"Pclass valores únicos: {sorted(df['Pclass'].unique())} -> {'OK' if set(df['Pclass'].unique()) == {1,2,3} else 'ERROR'}")

# Pasajeros con Fare = 0
fare_cero = df[df['Fare'] == 0]
print(f"\nPasajeros con Fare=0: {len(fare_cero)}")
if len(fare_cero) > 0:
    display(fare_cero[['PassengerId','Name','Pclass','Fare','Embarked']].head())

In [ ]:
# --- 5.7 Cardinalidad en categóricas ---
print("=== Cardinalidad de variables categóricas ===")
for c in ['Sex', 'Embarked', 'Cabin', 'Ticket']:
    n_unique = df[c].nunique()
    print(f"{c}: {n_unique} valores únicos")

print("\n--- Sex ---")
print(df['Sex'].value_counts())
print("\n--- Embarked ---")
print(df['Embarked'].value_counts(dropna=False))

**Conclusión Sección 5:**
- **Faltantes críticos:** Cabin (77.1%) — demasiados faltantes para usar directamente; Age (19.87%) — requiere imputación; Embarked (0.22%) — solo 2 valores faltantes.
- **Duplicados:** 0 filas duplicadas.
- **Rangos:** Age y Fare tienen rangos razonables. Hay 15 pasajeros con Fare=0 que podrían ser tripulación o errores de registro.
- **Tipos:** Los tipos están correctos. Pclass y Survived son numéricos pero representan categorías.
- **Decisión:** Se mantendrá Cabin como indicador (tiene/no tiene cabina). Embarked se imputará con la moda ('S'). Age se analizará con los valores disponibles.

## 6) Estadística descriptiva

In [ ]:
# ============================================================
# 6) Estadística descriptiva
# ============================================================

# Separar columnas numéricas relevantes (excluimos PassengerId que es solo ID)
num_cols = ['Age', 'Fare', 'SibSp', 'Parch', 'Pclass', 'Survived']
cat_cols = ['Sex', 'Embarked', 'Cabin', 'Name', 'Ticket']

# Variables de análisis (excluyendo PassengerId y Survived como binaria)
num_analisis = ['Age', 'Fare', 'SibSp', 'Parch']

print("Numéricas para análisis:", num_analisis)
print("Categóricas:", cat_cols[:3])

In [ ]:
def descriptivo_numerico(data: pd.DataFrame, cols: list) -> pd.DataFrame:
    out = []
    for c in cols:
        x = data[c].dropna()
        if x.empty:
            continue
        q = x.quantile([.10,.25,.50,.75,.90,.95,.99]).to_dict()
        # Moda
        moda = x.mode()
        moda_val = float(moda.iloc[0]) if not moda.empty else np.nan
        out.append({
            "col": c,
            "n": int(x.shape[0]),
            "mean": round(float(x.mean()), 4),
            "median": round(float(x.median()), 4),
            "moda": round(moda_val, 4),
            "std": round(float(x.std(ddof=1)), 4),
            "var": round(float(x.var(ddof=1)), 4),
            "min": round(float(x.min()), 4),
            "p10": round(float(q.get(0.10)), 4),
            "p25": round(float(q.get(0.25)), 4),
            "p50": round(float(q.get(0.50)), 4),
            "p75": round(float(q.get(0.75)), 4),
            "p90": round(float(q.get(0.90)), 4),
            "p95": round(float(q.get(0.95)), 4),
            "p99": round(float(q.get(0.99)), 4),
            "max": round(float(x.max()), 4),
            "iqr": round(float(q.get(0.75) - q.get(0.25)), 4),
            "rango": round(float(x.max() - x.min()), 4),
            "skew": round(float(stats.skew(x, bias=False)), 4) if len(x) > 2 else np.nan,
            "kurtosis": round(float(stats.kurtosis(x, bias=False)), 4) if len(x) > 3 else np.nan,
            "cv": round(float(x.std(ddof=1) / x.mean() * 100), 2) if x.mean() != 0 else np.nan,
        })
    return pd.DataFrame(out).set_index("col")

desc_num = descriptivo_numerico(df, num_analisis)
print("=== Estadística descriptiva — Variables numéricas ===")
display(desc_num)

In [ ]:
# Categóricas: value_counts
print("=== Estadística descriptiva — Variables categóricas ===")
for c in ['Sex', 'Embarked', 'Pclass', 'Survived']:
    print(f"\n--- {c} ---")
    vc = df[c].value_counts(dropna=False)
    pct = (vc / len(df) * 100).round(2)
    display(pd.DataFrame({"conteo": vc, "porcentaje": pct}))

**Conclusión Sección 6:**
- **Age:** Media 29.7 años, mediana 28. Distribución ligeramente sesgada a la derecha (skew=0.39). Rango amplio: 0.42–80 años.
- **Fare:** Alta asimetría (skew=4.79) y curtosis (33.40), indicando cola pesada a la derecha. La mediana (14.45) es mucho menor que la media (32.20), confirmando outliers altos. Coeficiente de variación muy alto (154%).
- **SibSp y Parch:** Mayoría de pasajeros viajaban solos (mediana 0 en ambas). Distribución muy sesgada.
- **Sobrevivencia:** Solo 38.4% sobrevivió — clases desbalanceadas.
- **Sexo:** 64.8% hombres vs 35.2% mujeres.
- **Embarque:** 72.3% embarcó en Southampton.

### 6.1 Descriptivo por grupos

In [ ]:
# ============================================================
# 6.1) Descriptivo por grupos
# ============================================================
# Se analizan múltiples agrupaciones relevantes para el Titanic

vars_agg = ['Age', 'Fare', 'SibSp', 'Parch']

# --- Grupo 1: Por Sexo (Sex) ---
print("=" * 60)
print("GRUPO 1: Estadísticas por SEXO (Sex)")
print("=" * 60)
agg_sex = df.groupby("Sex")[vars_agg].agg(["count", "mean", "median", "std", "min", "max"])
display(agg_sex)

# Tasa de supervivencia por sexo
print("\nTasa de supervivencia por sexo:")
display(df.groupby("Sex")["Survived"].agg(["count", "sum", "mean"]).rename(
    columns={"sum": "sobrevivieron", "mean": "tasa_supervivencia"}))

In [ ]:
# --- Grupo 2: Por Clase (Pclass) ---
print("=" * 60)
print("GRUPO 2: Estadísticas por CLASE (Pclass)")
print("=" * 60)
agg_pclass = df.groupby("Pclass")[vars_agg].agg(["count", "mean", "median", "std", "min", "max"])
display(agg_pclass)

# Tasa de supervivencia por clase
print("\nTasa de supervivencia por clase:")
display(df.groupby("Pclass")["Survived"].agg(["count", "sum", "mean"]).rename(
    columns={"sum": "sobrevivieron", "mean": "tasa_supervivencia"}))

In [ ]:
# --- Grupo 3: Por Puerto de embarque (Embarked) ---
print("=" * 60)
print("GRUPO 3: Estadísticas por PUERTO DE EMBARQUE (Embarked)")
print("=" * 60)
agg_emb = df.groupby("Embarked")[vars_agg].agg(["count", "mean", "median", "std"])
display(agg_emb)

print("\nTasa de supervivencia por puerto:")
display(df.groupby("Embarked")["Survived"].agg(["count", "sum", "mean"]).rename(
    columns={"sum": "sobrevivieron", "mean": "tasa_supervivencia"}))

In [ ]:
# --- Grupo 4: Por Supervivencia (Survived) ---
print("=" * 60)
print("GRUPO 4: Estadísticas por SUPERVIVENCIA (Survived)")
print("=" * 60)
agg_surv = df.groupby("Survived")[vars_agg].agg(["count", "mean", "median", "std", "min", "max"])
display(agg_surv)

In [ ]:
# --- Grupo 5: Cruzado Sex x Pclass ---
print("=" * 60)
print("GRUPO 5: Tasa de supervivencia cruzada SEX x PCLASS")
print("=" * 60)
cross = df.groupby(["Sex", "Pclass"])["Survived"].agg(["count", "sum", "mean"]).rename(
    columns={"sum": "sobrevivieron", "mean": "tasa_supervivencia"})
display(cross)

# Tabla pivote de supervivencia
print("\nTabla pivote: Tasa de supervivencia")
pivot = df.pivot_table(values="Survived", index="Sex", columns="Pclass", aggfunc="mean")
display(pivot.round(3))

In [ ]:
# --- Grupo 6: Por grupo de edad (creado con pd.qcut) ---
print("=" * 60)
print("GRUPO 6: Estadísticas por GRUPO DE EDAD (cuartiles)")
print("=" * 60)
df["Age_Group"] = pd.qcut(df["Age"], q=4, labels=["Joven", "Adulto_Joven", "Adulto", "Mayor"])
agg_age = df.groupby("Age_Group", observed=True).agg(
    n=("Survived", "count"),
    tasa_supervivencia=("Survived", "mean"),
    fare_media=("Fare", "mean"),
    fare_mediana=("Fare", "median")
).round(3)
display(agg_age)

**Conclusión Sección 6.1:**
- **Por sexo:** Las mujeres tenían tasa de supervivencia del 74.2% vs 18.9% de hombres — política de "mujeres y niños primero".
- **Por clase:** 1ra clase: 63.0%, 2da: 47.3%, 3ra: 24.2% — clara ventaja socioeconómica.
- **Por embarque:** Cherbourg (C) tuvo la mayor tasa (55.4%), posiblemente por mayor proporción de 1ra clase.
- **Cruzado Sex x Pclass:** Casi todas las mujeres de 1ra y 2da clase sobrevivieron (96.8% y 92.1%). Los hombres de 3ra clase tuvieron la menor tasa (13.5%).
- **Por edad:** Los jóvenes (0-20 años) tuvieron mayor supervivencia (44.4%), probablemente por inclusión de niños.
- **Decisión:** Sex, Pclass y Age son las variables más predictivas de supervivencia.

## 7) Visualización: básica y compleja

In [ ]:
def savefig(nombre: str):
    path = os.path.join("exports", "figuras", nombre)
    plt.tight_layout()
    plt.savefig(path, dpi=160, bbox_inches="tight")
    print("Guardado:", path)

In [ ]:
# --- 7.1 Histogramas + KDE (≥3 variables) ---
print("=== Histogramas + KDE ===")
for c in ['Age', 'Fare', 'SibSp']:
    plt.figure(figsize=(10, 5))
    sns.histplot(df[c].dropna(), kde=True, bins=30, color='steelblue')
    plt.axvline(df[c].mean(), color='red', linestyle='--', label=f'Media: {df[c].mean():.2f}')
    plt.axvline(df[c].median(), color='green', linestyle='--', label=f'Mediana: {df[c].median():.2f}')
    plt.legend()
    plt.title(f"Hist + KDE: {c}")
    plt.xlabel(c)
    plt.ylabel("Frecuencia")
    savefig(f"hist_kde_{c}.png")
    plt.show()

In [ ]:
# --- 7.2 ECDF (≥2 variables) ---
print("=== ECDF ===")
for c in ['Age', 'Fare']:
    plt.figure(figsize=(10, 5))
    sns.ecdfplot(data=df, x=c)
    plt.title(f"ECDF: {c}")
    plt.xlabel(c)
    plt.ylabel("Proporción acumulada")
    savefig(f"ecdf_{c}.png")
    plt.show()

In [ ]:
# --- 7.3 Boxplot por grupos ---
print("=== Boxplots por grupo ===")
# Age por Sex
plt.figure(figsize=(8, 5))
sns.boxplot(data=df, x="Sex", y="Age", palette="Set2")
plt.title("Boxplot: Age por Sex")
savefig("box_Age_by_Sex.png")
plt.show()

# Fare por Pclass
plt.figure(figsize=(8, 5))
sns.boxplot(data=df, x="Pclass", y="Fare", palette="Set2")
plt.title("Boxplot: Fare por Pclass")
savefig("box_Fare_by_Pclass.png")
plt.show()

# Age por Survived
plt.figure(figsize=(8, 5))
sns.boxplot(data=df, x="Survived", y="Age", palette="Set2")
plt.title("Boxplot: Age por Survived")
savefig("box_Age_by_Survived.png")
plt.show()

In [ ]:
# --- 7.4 Violin plots por grupos ---
print("=== Violin plots ===")
# Age por Sex
plt.figure(figsize=(8, 5))
sns.violinplot(data=df, x="Sex", y="Age", palette="muted", cut=0)
plt.title("Violin: Age por Sex")
savefig("violin_Age_by_Sex.png")
plt.show()

# Fare por Pclass
plt.figure(figsize=(8, 5))
sns.violinplot(data=df, x="Pclass", y="Fare", palette="muted", cut=0)
plt.title("Violin: Fare por Pclass")
savefig("violin_Fare_by_Pclass.png")
plt.show()

# Age por Survived
plt.figure(figsize=(8, 5))
sns.violinplot(data=df, x="Survived", y="Age", hue="Sex", split=True, palette="Set1", cut=0)
plt.title("Violin: Age por Survived, dividido por Sex")
savefig("violin_Age_Survived_Sex.png")
plt.show()

In [ ]:
# --- 7.5 Scatter plots (≥3 pares) ---
print("=== Scatter plots ===")

# Age vs Fare (coloreado por Survived)
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df, x="Age", y="Fare", hue="Survived", alpha=0.6, palette="coolwarm")
plt.title("Scatter: Age vs Fare (coloreado por Survived)")
savefig("scatter_Age_vs_Fare.png")
plt.show()

# Age vs SibSp
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df, x="Age", y="SibSp", hue="Pclass", alpha=0.6, palette="viridis")
plt.title("Scatter: Age vs SibSp (coloreado por Pclass)")
savefig("scatter_Age_vs_SibSp.png")
plt.show()

# Fare vs Parch
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df, x="Fare", y="Parch", hue="Survived", alpha=0.6, palette="coolwarm")
plt.title("Scatter: Fare vs Parch (coloreado por Survived)")
savefig("scatter_Fare_vs_Parch.png")
plt.show()

In [ ]:
# --- 7.6 Pairplot (subset) ---
print("=== Pairplot ===")
subset_cols = ['Age', 'Fare', 'SibSp', 'Parch', 'Survived']
g = sns.pairplot(df[subset_cols].dropna(), hue="Survived", palette="coolwarm",
                 diag_kind="kde", plot_kws={"alpha": 0.5})
g.fig.suptitle("Pairplot: Variables numéricas por Survived", y=1.02)
g.savefig(os.path.join("exports", "figuras", "pairplot_subset.png"), dpi=160, bbox_inches="tight")
plt.show()

In [ ]:
# --- 7.7 Heatmap de correlación ---
print("=== Heatmap de correlación ===")
corr_cols = ['Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare']
corr = df[corr_cols].corr()
plt.figure(figsize=(10, 8))
mask = np.triu(np.ones_like(corr, dtype=bool))  # Máscara triangular
sns.heatmap(corr, annot=True, fmt=".3f", cmap="RdBu_r", center=0,
            mask=mask, square=True, linewidths=0.5)
plt.title("Heatmap de correlación (Pearson)")
savefig("heatmap_correlacion.png")
plt.show()

In [ ]:
# --- 7.8 Visualización avanzada 1: Jointplot ---
print("=== Jointplot ===")
jp = sns.jointplot(data=df, x="Age", y="Fare", kind="hex", height=8,
                   color="steelblue", marginal_kws={"bins": 30})
jp.fig.suptitle("Jointplot: Age vs Fare (hexbin)", y=1.02)
jp.fig.savefig(os.path.join("exports", "figuras", "jointplot_Age_vs_Fare.png"),
               dpi=160, bbox_inches="tight")
plt.show()

In [ ]:
# --- 7.9 Visualización avanzada 2: FacetGrid ---
print("=== FacetGrid: Distribución de Age por Pclass y Survived ===")
g = sns.FacetGrid(df.dropna(subset=["Age"]), col="Pclass", hue="Survived",
                  palette="coolwarm", height=4, aspect=1.2)
g.map(sns.histplot, "Age", kde=True, bins=20, alpha=0.6)
g.add_legend(title="Survived")
g.fig.suptitle("Distribución de Age por Pclass y Survived", y=1.02)
g.savefig(os.path.join("exports", "figuras", "facetgrid_age_pclass_survived.png"),
          dpi=160, bbox_inches="tight")
plt.show()

In [ ]:
# --- 7.10 Visualización avanzada 3: Countplot de supervivencia ---
print("=== Countplots de supervivencia ===")
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Por Sex
sns.countplot(data=df, x="Sex", hue="Survived", palette="coolwarm", ax=axes[0])
axes[0].set_title("Supervivencia por Sexo")

# Por Pclass
sns.countplot(data=df, x="Pclass", hue="Survived", palette="coolwarm", ax=axes[1])
axes[1].set_title("Supervivencia por Clase")

# Por Embarked
sns.countplot(data=df, x="Embarked", hue="Survived", palette="coolwarm", ax=axes[2])
axes[2].set_title("Supervivencia por Puerto")

plt.tight_layout()
savefig("countplot_supervivencia.png")
plt.show()

In [ ]:
# --- 7.11 Visualización avanzada 4: Heatmap supervivencia Sex x Pclass ---
print("=== Heatmap de tasa de supervivencia ===")
pivot_surv = df.pivot_table(values="Survived", index="Sex", columns="Pclass", aggfunc="mean")
plt.figure(figsize=(8, 4))
sns.heatmap(pivot_surv, annot=True, fmt=".2%", cmap="RdYlGn", vmin=0, vmax=1,
            linewidths=1, square=True)
plt.title("Tasa de supervivencia: Sex x Pclass")
savefig("heatmap_supervivencia_sex_pclass.png")
plt.show()

**Conclusión Sección 7:**
- **Hist/KDE:** Age tiene distribución relativamente normal con un pico en 20-30 años. Fare está fuertemente sesgada a la derecha con outliers extremos.
- **Boxplots:** Fare por Pclass muestra clara diferencia entre clases; 1ra clase tiene tarifas mucho más altas y dispersas.
- **Violin (dividido):** Las mujeres sobrevivientes se distribuyeron uniformemente en edad, mientras que los hombres sobrevivientes tienden a ser más jóvenes.
- **Scatter:** No hay correlación lineal fuerte entre Age y Fare, pero se observan patrones de supervivencia.
- **Correlación:** La correlación más fuerte con Survived es Pclass (-0.34) y Fare (0.26). Pclass y Fare están inversamente correlacionados (-0.55).
- **FacetGrid:** En 3ra clase la mortalidad fue alta para todas las edades. En 1ra clase hubo alta supervivencia especialmente en mujeres.
- **Decisión:** Las visualizaciones confirman que Sex, Pclass y Fare son las variables más discriminantes para predecir supervivencia.

## 8) Outliers: univariados y multivariados

In [ ]:
# ============================================================
# 8) Outliers
# ============================================================

def iqr_fences(x: pd.Series, k=1.5):
    q1, q3 = x.quantile(0.25), x.quantile(0.75)
    iqr = q3 - q1
    return q1 - k*iqr, q3 + k*iqr, iqr

def modified_zscore_mad(x: pd.Series):
    x = x.dropna()
    med = x.median()
    mad = np.median(np.abs(x - med))
    if mad == 0:
        return pd.Series(np.zeros(len(x)), index=x.index)
    return 0.6745 * (x - med) / mad

# Usamos variables significativas (NO PassengerId)
CANDIDATAS = ['Age', 'Fare', 'SibSp', 'Parch']
OUTLIER_RES = {}

for c in CANDIDATAS:
    x = df[c].dropna()
    if x.empty:
        continue

    lo15, hi15, _ = iqr_fences(x, k=1.5)
    lo30, hi30, _ = iqr_fences(x, k=3.0)
    idx_iqr15 = x[(x < lo15) | (x > hi15)].index
    idx_iqr30 = x[(x < lo30) | (x > hi30)].index

    z = (x - x.mean()) / x.std(ddof=1) if x.std(ddof=1) != 0 else pd.Series(np.zeros(len(x)), index=x.index)
    idx_z3 = z[abs(z) > 3].index

    mz = modified_zscore_mad(x)
    idx_mz35 = mz[abs(mz) > 3.5].index

    OUTLIER_RES[c] = {
        "iqr_1.5": len(idx_iqr15),
        "iqr_3.0": len(idx_iqr30),
        "z_gt_3": len(idx_z3),
        "modz_gt_3.5": len(idx_mz35)
    }

print("=== Detección de outliers univariados ===")
outlier_df = pd.DataFrame(OUTLIER_RES).T
outlier_df = outlier_df.sort_values("iqr_1.5", ascending=False)
display(outlier_df)

In [ ]:
# Visualizar outliers de Fare (variable con más outliers)
print("=== Outliers de Fare ===")
x_fare = df['Fare'].dropna()
lo15, hi15, iqr_val = iqr_fences(x_fare, k=1.5)
lo30, hi30, _ = iqr_fences(x_fare, k=3.0)

print(f"IQR: {iqr_val:.2f}")
print(f"Cercas 1.5*IQR: [{lo15:.2f}, {hi15:.2f}]")
print(f"Cercas 3.0*IQR: [{lo30:.2f}, {hi30:.2f}]")
print(f"\nOutliers extremos (Fare > {hi30:.2f}):")
display(df[df['Fare'] > hi30][['PassengerId', 'Name', 'Pclass', 'Fare', 'Survived']].head(10))

In [ ]:
# --- Outliers multivariados ---
print("=== Detección de outliers multivariados ===")

# Usamos solo variables numéricas relevantes
X = df[['Age', 'Fare', 'SibSp', 'Parch', 'Pclass']].copy()
X = X.replace([np.inf, -np.inf], np.nan)
X = X.fillna(X.median(numeric_only=True))

pt = PowerTransformer(method="yeo-johnson", standardize=True)
X_t = pt.fit_transform(X)

# DBSCAN
db = DBSCAN(eps=1.8, min_samples=10)
labels = db.fit_predict(X_t)
idx_dbscan_noise = np.where(labels == -1)[0]

# Isolation Forest
iso = IsolationForest(n_estimators=300, contamination="auto", random_state=42)
pred_iso = iso.fit_predict(X_t)
idx_iso = np.where(pred_iso == -1)[0]

# LOF
lof = LocalOutlierFactor(n_neighbors=25, contamination="auto")
pred_lof = lof.fit_predict(X_t)
idx_lof = np.where(pred_lof == -1)[0]

print(f"DBSCAN (noise): {len(idx_dbscan_noise)} outliers")
print(f"IsolationForest: {len(idx_iso)} outliers")
print(f"LOF: {len(idx_lof)} outliers")

# Sistema de votación
flags = pd.DataFrame({"dbscan_noise": 0, "iso_outlier": 0, "lof_outlier": 0}, index=df.index)
flags.iloc[idx_dbscan_noise, 0] = 1
flags.iloc[idx_iso, 1] = 1
flags.iloc[idx_lof, 2] = 1
flags["votes"] = flags.sum(axis=1)

print("\n=== Sistema de votación (# métodos que detectan outlier) ===")
display(flags["votes"].value_counts().sort_index(ascending=False))

print("\n=== Top 15 outliers multivariados (detectados por 2+ métodos) ===")
top_outliers = flags[flags["votes"] >= 2].sort_values("votes", ascending=False)
display(df.loc[top_outliers.head(15).index, ['PassengerId', 'Name', 'Pclass', 'Sex', 'Age', 'Fare', 'SibSp', 'Parch']])

**Conclusión Sección 8:**
- **Fare** es la variable con más outliers (116 por IQR 1.5x). Los valores extremos (>300 libras) corresponden a pasajeros de 1ra clase en suites de lujo — son **casos raros válidos**, no errores.
- **SibSp y Parch** tienen outliers por familias grandes: son **eventos reales** (familias numerosas que viajaban juntas).
- **Age** tiene pocos outliers. El valor máximo (80 años) es válido.
- **Multivariado:** Los outliers consensuados (2+ métodos) son principalmente pasajeros con familias muy grandes y tarifas extremas.
- **Decisión:** No eliminar outliers porque representan eventos reales. Para modelado, se recomienda winsorizar Fare y transformar con log.

## 9) Tratamiento de outliers (antes vs. después)

In [ ]:
# ============================================================
# 9) Tratamiento de outliers — Variable: Fare
# ============================================================
# Se elige Fare por ser la variable con mayor asimetría (skew=4.79) y más outliers

VAR_TRATAR = "Fare"
x0 = df[VAR_TRATAR].copy()

# Tratamiento 1: Winsorización al 1%
x_w = pd.Series(winsorize(x0, limits=[0.01, 0.01]), index=x0.index)

# Tratamiento 2: Transformación log1p (Fare >= 0)
x_log = np.log1p(x0)

# Tratamiento 3: Transformación Yeo-Johnson
pt_fare = PowerTransformer(method="yeo-johnson", standardize=True)
x_yj = pd.Series(pt_fare.fit_transform(x0.values.reshape(-1, 1)).flatten(), index=x0.index)

def resumen_simple(s: pd.Series) -> pd.Series:
    s2 = s.dropna()
    return pd.Series({
        "n": int(s2.shape[0]),
        "mean": round(float(s2.mean()), 4),
        "median": round(float(s2.median()), 4),
        "std": round(float(s2.std(ddof=1)), 4),
        "min": round(float(s2.min()), 4),
        "p25": round(float(s2.quantile(0.25)), 4),
        "p75": round(float(s2.quantile(0.75)), 4),
        "max": round(float(s2.max()), 4),
        "skew": round(float(stats.skew(s2, bias=False)), 4) if s2.shape[0] > 2 else np.nan,
        "kurtosis": round(float(stats.kurtosis(s2, bias=False)), 4) if s2.shape[0] > 3 else np.nan,
    })

comparacion = pd.DataFrame({
    "original": resumen_simple(x0),
    "winsor_1pct": resumen_simple(x_w),
    "log1p": resumen_simple(x_log),
    "yeo_johnson": resumen_simple(x_yj)
})

print("=== Comparación antes vs. después del tratamiento ===")
display(comparacion)

In [ ]:
# --- Gráficos comparativos ---
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Original
sns.histplot(x0, kde=True, bins=30, ax=axes[0,0], color="steelblue")
axes[0,0].set_title(f"Original: {VAR_TRATAR} (skew={stats.skew(x0, bias=False):.2f})")

# Winsor
sns.histplot(x_w, kde=True, bins=30, ax=axes[0,1], color="orange")
axes[0,1].set_title(f"Winsor 1%: {VAR_TRATAR} (skew={stats.skew(x_w, bias=False):.2f})")

# Log1p
sns.histplot(x_log, kde=True, bins=30, ax=axes[1,0], color="green")
axes[1,0].set_title(f"log1p: {VAR_TRATAR} (skew={stats.skew(x_log, bias=False):.2f})")

# Yeo-Johnson
sns.histplot(x_yj, kde=True, bins=30, ax=axes[1,1], color="purple")
axes[1,1].set_title(f"Yeo-Johnson: {VAR_TRATAR} (skew={stats.skew(x_yj, bias=False):.2f})")

plt.suptitle(f"Tratamiento de outliers: {VAR_TRATAR}", fontsize=14, y=1.02)
plt.tight_layout()
savefig(f"tratamiento_comparacion_{VAR_TRATAR}.png")
plt.show()

In [ ]:
# --- Boxplots antes vs. después ---
fig, axes = plt.subplots(1, 4, figsize=(16, 5))

sns.boxplot(y=x0, ax=axes[0], color="steelblue")
axes[0].set_title("Original")

sns.boxplot(y=x_w, ax=axes[1], color="orange")
axes[1].set_title("Winsor 1%")

sns.boxplot(y=x_log, ax=axes[2], color="green")
axes[2].set_title("log1p")

sns.boxplot(y=x_yj, ax=axes[3], color="purple")
axes[3].set_title("Yeo-Johnson")

plt.suptitle(f"Boxplots antes vs. después: {VAR_TRATAR}", fontsize=14)
plt.tight_layout()
savefig(f"boxplot_tratamiento_{VAR_TRATAR}.png")
plt.show()

**Conclusión Sección 9:**
- **Original:** Fare tiene skewness=4.79 y kurtosis=33.40 — distribución muy asimétrica.
- **Winsorización 1%:** Reduce el skewness moderadamente al recortar los extremos, preservando la escala original. Útil cuando se necesita interpretar los valores en la escala original.
- **log1p:** Reduce drásticamente el skewness, acercando la distribución a la normalidad. Ideal para modelos que asumen normalidad.
- **Yeo-Johnson:** El mejor resultado en términos de simetría (skew ≈ 0), pero pierde interpretabilidad.
- **Decisión:** Para análisis descriptivo usar Winsorización. Para modelado predictivo usar log1p por su buen balance entre normalización e interpretabilidad.

## 10) Inferencia estadística mínima

In [ ]:
# ============================================================
# 10) Inferencia estadística
# ============================================================
ALPHA = 0.05

# --- 10.1 Prueba de normalidad: Shapiro-Wilk sobre Age ---
print("=" * 60)
print("10.1 PRUEBA DE NORMALIDAD — Shapiro-Wilk")
print("=" * 60)

var_norm = "Age"
x = df[var_norm].dropna()
x_sample = x.sample(min(len(x), 5000), random_state=42)

W, p = stats.shapiro(x_sample)
print(f"Variable: {var_norm}")
print(f"H0: La variable {var_norm} sigue una distribución normal")
print(f"H1: La variable {var_norm} NO sigue una distribución normal")
print(f"α = {ALPHA}")
print(f"Resultado: W = {W:.4f}, p-value = {p:.6f}")
print(f"Conclusión: {'Rechazamos H0 — Age NO es normal (p < α)' if p < ALPHA else 'No rechazamos H0 — Age es compatible con normalidad'}")

# También para Fare
var_norm2 = "Fare"
x2 = df[var_norm2].dropna()
W2, p2 = stats.shapiro(x2.sample(min(len(x2), 5000), random_state=42))
print(f"\nShapiro-Wilk sobre Fare: W={W2:.4f}, p={p2:.6f}")
print(f"Conclusión: {'Rechazamos H0 — Fare NO es normal' if p2 < ALPHA else 'No rechazamos H0'}")

In [ ]:
# QQ-plots para verificar visualmente normalidad
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

stats.probplot(df['Age'].dropna(), dist="norm", plot=axes[0])
axes[0].set_title("QQ-Plot: Age")

stats.probplot(df['Fare'].dropna(), dist="norm", plot=axes[1])
axes[1].set_title("QQ-Plot: Fare")

plt.tight_layout()
savefig("qqplot_age_fare.png")
plt.show()

In [ ]:
# --- 10.2 t-test (Welch) — Comparar Fare entre sobrevivientes y no sobrevivientes ---
print("=" * 60)
print("10.2 T-TEST DE WELCH — Fare por Survived")
print("=" * 60)

grupo_a = df.loc[df["Survived"] == 1, "Fare"].dropna()
grupo_b = df.loc[df["Survived"] == 0, "Fare"].dropna()

print(f"H0: No hay diferencia significativa en Fare entre sobrevivientes y no sobrevivientes")
print(f"H1: Hay diferencia significativa en Fare entre ambos grupos")
print(f"α = {ALPHA}")
print(f"\nSobrevivientes (n={len(grupo_a)}): media={grupo_a.mean():.2f}, mediana={grupo_a.median():.2f}")
print(f"No sobrevivientes (n={len(grupo_b)}): media={grupo_b.mean():.2f}, mediana={grupo_b.median():.2f}")

t_stat, p_ttest = stats.ttest_ind(grupo_a, grupo_b, equal_var=False)
print(f"\nResultado: t = {t_stat:.4f}, p-value = {p_ttest:.6f}")
print(f"Conclusión: {'Rechazamos H0 — Hay diferencia significativa en Fare' if p_ttest < ALPHA else 'No rechazamos H0'}")
print(f"Interpretación: Los sobrevivientes pagaron tarifas significativamente más altas (media ${grupo_a.mean():.2f} vs ${grupo_b.mean():.2f}).")
print(f"Esto se explica porque los pasajeros de 1ra clase (tarifas altas) tuvieron mayor tasa de supervivencia.")

In [ ]:
# --- 10.3 ANOVA — Comparar Age entre las 3 clases (Pclass) ---
print("=" * 60)
print("10.3 ANOVA — Age por Pclass (3 grupos)")
print("=" * 60)

d_anova = df[["Pclass", "Age"]].dropna().copy()
d_anova["Pclass"] = d_anova["Pclass"].astype("category")

print(f"H0: La edad media es igual en las 3 clases (μ1 = μ2 = μ3)")
print(f"H1: Al menos una clase tiene una edad media diferente")
print(f"α = {ALPHA}")

# Estadísticas por grupo
print("\nEstadísticas por grupo:")
for cls in [1, 2, 3]:
    ages = d_anova.loc[d_anova["Pclass"] == cls, "Age"]
    print(f"  Clase {cls}: n={len(ages)}, media={ages.mean():.2f}, std={ages.std():.2f}")

# ANOVA con statsmodels
model = sm.formula.ols("Age ~ C(Pclass)", data=d_anova).fit()
tabla_anova = anova_lm(model, typ=2)
print("\nTabla ANOVA:")
display(tabla_anova)

p_anova = tabla_anova.loc["C(Pclass)", "PR(>F)"]
print(f"\np-value = {p_anova:.6f}")
print(f"Conclusión: {'Rechazamos H0 — La edad difiere significativamente entre clases' if p_anova < ALPHA else 'No rechazamos H0'}")
print(f"Interpretación: Los pasajeros de 1ra clase eran en promedio mayores que los de 3ra clase,")
print(f"lo cual es consistente con que personas de mayor edad tendían a tener mayor poder adquisitivo.")

In [ ]:
# --- 10.4 Correlación Pearson y Spearman — Age vs Fare ---
print("=" * 60)
print("10.4 CORRELACIÓN — Age vs Fare")
print("=" * 60)

df_corr = df[["Age", "Fare"]].dropna()

print(f"H0: No hay correlación lineal significativa entre Age y Fare (ρ = 0)")
print(f"H1: Existe correlación lineal significativa entre Age y Fare (ρ ≠ 0)")
print(f"α = {ALPHA}")

# Pearson
r_pearson, p_pearson = stats.pearsonr(df_corr["Age"], df_corr["Fare"])
print(f"\nPearson: r = {r_pearson:.4f}, p-value = {p_pearson:.6f}")
print(f"Conclusión Pearson: {'Rechazamos H0 — Correlación significativa' if p_pearson < ALPHA else 'No rechazamos H0 — No hay correlación lineal significativa'}")

# Spearman (no asume normalidad)
r_spearman, p_spearman = stats.spearmanr(df_corr["Age"], df_corr["Fare"])
print(f"\nSpearman: ρ = {r_spearman:.4f}, p-value = {p_spearman:.6f}")
print(f"Conclusión Spearman: {'Rechazamos H0 — Correlación monótona significativa' if p_spearman < ALPHA else 'No rechazamos H0'}")

print(f"\nInterpretación: La correlación entre Age y Fare es débil (r≈{r_pearson:.2f}).")
print(f"Aunque estadísticamente significativa (muestra grande), el tamaño del efecto es pequeño.")
print(f"La tarifa está más determinada por la clase del boleto que por la edad del pasajero.")

In [ ]:
# --- 10.5 Prueba Chi-cuadrado — Relación entre Sex y Survived (extra) ---
print("=" * 60)
print("10.5 CHI-CUADRADO — Sex vs Survived (prueba adicional)")
print("=" * 60)

tabla_contingencia = pd.crosstab(df["Sex"], df["Survived"])
print("Tabla de contingencia:")
display(tabla_contingencia)

chi2, p_chi, dof, expected = stats.chi2_contingency(tabla_contingencia)
print(f"\nH0: Sex y Survived son independientes")
print(f"H1: Existe asociación entre Sex y Survived")
print(f"α = {ALPHA}")
print(f"\nχ² = {chi2:.4f}, p-value = {p_chi:.10f}, grados de libertad = {dof}")
print(f"Conclusión: {'Rechazamos H0 — Existe asociación significativa entre Sex y Survived' if p_chi < ALPHA else 'No rechazamos H0'}")

# Coeficiente de Cramér V
n = tabla_contingencia.sum().sum()
cramers_v = np.sqrt(chi2 / (n * (min(tabla_contingencia.shape) - 1)))
print(f"Cramér's V = {cramers_v:.4f} (fuerza de la asociación: {'fuerte' if cramers_v > 0.3 else 'moderada' if cramers_v > 0.1 else 'débil'})")
print(f"\nInterpretación: El sexo del pasajero está fuertemente asociado con la supervivencia.")
print(f"Las mujeres tuvieron una probabilidad significativamente mayor de sobrevivir que los hombres.")

**Conclusión Sección 10:**
- **Normalidad:** Ni Age ni Fare siguen una distribución normal (Shapiro-Wilk p < 0.05). Esto justifica usar pruebas no paramétricas o Welch t-test (robusto a no-normalidad).
- **t-test Welch:** Los sobrevivientes pagaron tarifas significativamente más altas (p < 0.001). Esto se debe a que la clase alta tenía prioridad en los botes salvavidas.
- **ANOVA:** La edad difiere significativamente entre las 3 clases (p < 0.001). Los pasajeros de 1ra clase eran mayores en promedio.
- **Correlación:** La correlación Age-Fare es débil pero significativa. La tarifa depende más de la clase que de la edad.
- **Chi-cuadrado:** Fuerte asociación entre sexo y supervivencia (Cramér's V > 0.5), confirmando la política de "mujeres y niños primero".

## 11) Exportables y cierre

In [ ]:
# ============================================================
# 11) Exportables
# ============================================================
os.makedirs("exports", exist_ok=True)
os.makedirs("exports/figuras", exist_ok=True)

# 1) tabla_resumen.csv
desc_export = descriptivo_numerico(df, ['Age', 'Fare', 'SibSp', 'Parch', 'Pclass', 'Survived'])
desc_export.to_csv("exports/tabla_resumen.csv")
print("OK -> exports/tabla_resumen.csv")

# 2) tests.json
tests = {
    "alpha": ALPHA,
    "shapiro_age": {"variable": "Age", "W": float(W), "p": float(p), "normal": bool(p >= ALPHA)},
    "shapiro_fare": {"variable": "Fare", "W": float(W2), "p": float(p2), "normal": bool(p2 >= ALPHA)},
    "ttest_welch": {
        "variable": "Fare",
        "grupo": "Survived",
        "t": float(t_stat),
        "p": float(p_ttest),
        "significativo": bool(p_ttest < ALPHA)
    },
    "anova_age_pclass": {
        "variable": "Age",
        "grupo": "Pclass",
        "p": float(p_anova),
        "significativo": bool(p_anova < ALPHA)
    },
    "pearson_age_fare": {
        "r": float(r_pearson),
        "p": float(p_pearson),
        "significativo": bool(p_pearson < ALPHA)
    },
    "spearman_age_fare": {
        "rho": float(r_spearman),
        "p": float(p_spearman),
        "significativo": bool(p_spearman < ALPHA)
    },
    "chi2_sex_survived": {
        "chi2": float(chi2),
        "p": float(p_chi),
        "cramers_v": float(cramers_v),
        "significativo": bool(p_chi < ALPHA)
    }
}

with open("exports/tests.json", "w", encoding="utf-8") as f:
    json.dump(tests, f, ensure_ascii=False, indent=2)
print("OK -> exports/tests.json")

# 3) Listar figuras exportadas
figuras = os.listdir("exports/figuras")
print(f"\nFiguras exportadas ({len(figuras)}):")
for fig in sorted(figuras):
    print(f"  • exports/figuras/{fig}")

## Resumen ejecutivo

- **Dataset principal:** Titanic — Machine Learning from Disaster (Kaggle). 891 pasajeros, 12 variables. Enlace: https://www.kaggle.com/competitions/titanic

- **Unidad de observación:** Cada fila representa un pasajero del Titanic con sus características demográficas y de viaje.

- **Variables críticas:** Sex (género), Pclass (clase socioeconómica), Age (edad) y Fare (tarifa) son las más relevantes para predecir supervivencia.

- **Hallazgos de calidad:** Cabin tiene 77.1% de faltantes (casi inutilizable). Age tiene 19.87% faltante (requiere manejo cuidadoso). 15 registros con Fare=0 (posibles errores). Sin duplicados.

- **Hallazgos descriptivos:** Solo 38.4% sobrevivió. Las mujeres sobrevivieron significativamente más (74.2%) que los hombres (18.9%). La 1ra clase tuvo 63% de supervivencia vs 24.2% en 3ra clase. Fare tiene alta asimetría (skew=4.79) con outliers en tarifas de lujo.

- **Relaciones clave:** Pclass-Fare correlación fuerte negativa (-0.55). Sex-Survived asociación fuerte (Cramér's V > 0.5). La combinación Sex + Pclass predice supervivencia con alta precisión (mujeres 1ra clase: 96.8%).

- **Outliers:** Fare tiene los más outliers (116 por IQR). Son casos válidos (suites de lujo), no errores. SibSp/Parch tienen outliers por familias grandes.

- **Tratamiento y efecto:** Winsorización al 1% y log1p reducen significativamente la asimetría de Fare. log1p es el mejor balance normalización-interpretabilidad.

- **Inferencia:** Todas las pruebas resultaron significativas (α=0.05): diferencia en Fare por supervivencia (t-test), diferencia en Age por clase (ANOVA), asociación Sex-Survived (chi-cuadrado). Age y Fare no son normales (Shapiro-Wilk).

- **Limitaciones:** Muestra de solo 891 de 2224 pasajeros (no representativa de toda la tripulación). Cabin con demasiados faltantes. Age faltante en 20% puede sesgar análisis de edad. No se dispone de información sobre ubicación en el barco al momento del impacto.

---

## Rúbrica (100%)

| Criterio | Peso | Evidencia |
|----------|------|-----------|
| EDA descriptivo (tablas + interpretación) | 30% | Sección 6 y 6.1: descriptivo completo con media/mediana/moda/varianza/std/IQR/cuantiles/skew/kurtosis + agrupaciones por Sex, Pclass, Embarked, Survived, Sex×Pclass, y grupo de edad |
| Visualización (básica + compleja) | 20% | Sección 7: Hist+KDE, ECDF, Boxplot, Violin, Scatter, Pairplot, Heatmap correlación, Jointplot, FacetGrid, Countplot, Heatmap supervivencia (>10 gráficos) |
| Outliers (multi-método + discusión) | 20% | Sección 8: IQR (1.5x y 3x), z-score, modified z-score MAD, DBSCAN, Isolation Forest, LOF + clasificación (error/raro/real) |
| Tratamiento (antes/después + justificación) | 15% | Sección 9: Winsorización + log1p + Yeo-Johnson con tabla comparativa y gráficos |
| Reporte + reproducibilidad + exportables | 15% | Sección 11: tabla_resumen.csv, tests.json, >10 figuras. Conclusiones en cada sección. Semilla fija (random_state=42) |